In [ ]:
import numpy as np
import os
import sys 
import types

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.animation as manimation

from IPython.display import HTML, Video

In [ ]:
# help with Illustrator/inkscape fonts
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['ps.fonttype'] = 42

# Notebook setup (path trick) and local import

In [ ]:
PACKAGE_ROOT = os.path.dirname(os.path.abspath(''))
print(PACKAGE_ROOT)
sys.path.append(PACKAGE_ROOT)

In [ ]:
from load_sweep import wrapper_load_or_digest_sweep, NAMES_IMPORTANT_GRAPHS

from settings import DIR_OUTPUT
from utils_io import pickle_load
from utils_networkx import check_tree_isomorphism, draw_from_adjacency, check_tree_isomorphism_with_insect

## Specify output directory

In [ ]:
NB_OUTPUT = PACKAGE_ROOT + os.sep + 'notebooks' + os.sep + 'figures_regenerate'
if not os.path.exists(NB_OUTPUT):
    os.makedirs(NB_OUTPUT)

## Video Functions

In [ ]:
def setup_ffmpeg_writer(fps=15, bitrate=None, title='Sample movie', artist='Matplotlib', comment='comment here'):
    # Define the meta data for the movie
    FFMpegWriter = manimation.writers['ffmpeg']
    metadata = dict(title=title, artist=artist, comment=comment)
    writer = FFMpegWriter(fps=fps, bitrate=bitrate, metadata=metadata)
    return writer

def vid_fpath_to_html(fpath):
    """
    - Show video using
        HTML(s)
    - Need to wrap it like this because it doesn't like abspath
    - Alternate method for absolute path:
        Video(fpath)
    """
    s = f"""
        <div align="middle">
        <video width="80%" controls>
              <source src={fpath.split("notebooks")[1][1:]} type="video/mp4">
        </video></div>"""
    return s

# Figure 4B - 3D param space of networks

### Curated points in regions of parameter space (Text Fig. 4a)

In [ ]:
# size: left is ONE PARTICULAR region, right is ALL regions

size_data = np.array([[1235204,1236138],
[145178,288215],
[177422,446894],
[123143,131322],
[231680,231680],
[17,17],
[81884,124822],
[120,1021],
[45,60],
[13372,79015],
[274,3319],
[365,1954],
[404,898],
[231,806],
[249,348],
[76,164],
[14,61],
[133,193],
[265,315],
[200,241],
[125,161],
[10622,10622],
[7,11],
[14235,17408],
[9644,14926],
[5876,6018]])


def linear_str(a):
    return r'$%d$ (linear)' % a

curated_regions = {
    'P. communis 1': {'pt': [4.77987421383648,0.106414141414141,-0.00499999999999999],
                      'size': [1235204,1236138],
                      'str': r'$\mathit{P. communis}$'},
    'P. sauteri 8': {'pt': [1.61006289308176,0.0833838383838384,0.061],
                     'size': [145178,288215],
                     'str': r'$\mathit{P. sauteri}$'},
    'O. labronica 2': {'pt': [1.76100628930818,0.120808080808081,-0.057],
                       'size': [177422,446894],
                       'str': r'$\mathit{O. labronica}$'},
    'D. melanogaster 1': {'pt': [5.38364779874214,0.0805050505050505,0],
                          'size': [123143,131322],
                          'str': r'$\mathit{D. melanogaster}$'},
    'one cell': {'pt': [4.0251572327044,0.0488383838383838,0],
                 'size': [231680,231680],      
                 'str': 'One cell'},
    'N. vitripennis': {'pt': [0.150943396226415,0.0613131313131313,0.026],
                       'size': [17,17],
                       'str': r'$\mathit{N. vitripennis}$'},
    'G. natator 4': {'pt': [6.33962264150943,0.0776262626262626,0.033],
                     'size': [81884,124822],
                     'str': r'$\mathit{G. natator}$'},
    'C. perla main 4': {'pt': [0.20125786163522,0.0718686868686869,-0.00899999999999999],
                        'size': [120,1021],
                        'str': r'$\mathit{C. perla}$ (12 cell)'},
    'C. perla secondary 1': {'pt': [0.10062893081761,0.0747474747474748,-0.018],
                             'size': [45,60],
                             'str': r'$\mathit{C. perla}$ (13 cell alt.)'},
    '5-cell linear 2': {'pt': [0.553459119496855,0.0632323232323232,-0.017],
                        'size': [13372,79015],
                        'str': linear_str(5)},
    '6-cell linear 15': {'pt': [0.452830188679245,0.0728282828282828,0.00600000000000001],
                         'size': [274,3319],
                         'str': linear_str(6)},
    '7-cell linear 17': {'pt': [0.30188679245283,0.0651515151515152,0.003],
                         'size': [365,1954],
                         'str': linear_str(7)},
    '8-cell linear 2': {'pt': [0.251572327044025,0.0824242424242424,-0.083],
                        'size': [404,898],
                        'str': linear_str(8)},
    '9-cell linear 5': {'pt': [0.251572327044025,0.0661111111111111,-0.011],
                        'size': [231,806],
                        'str': linear_str(9)},
    '10-cell linear 2': {'pt': [0.20125786163522,0.0939393939393939,-0.085],
                         'size': [249,348],
                         'str': linear_str(10)},
    '11-cell linear 1': {'pt': [0.150943396226415,0.0996969696969697,-0.087],
                         'size': [76,164],
                         'str': linear_str(11)},
    '12-cell linear 1': {'pt': [0.150943396226415,0.0584343434343434,0.042],
                         'size': [14,61],
                         'str': linear_str(12)},
    '13-cell linear 7': {'pt': [0.251572327044025,0.0555555555555556,0.089],
                         'size': [133,193],
                         'str': linear_str(13)},
    '14-cell linear 9': {'pt': [0.30188679245283,0.0555555555555556,0.084],
                         'size': [265,315],
                         'str': linear_str(14)},
    '15-cell linear 4': {'pt': [0.30188679245283,0.0555555555555556,0.077],
                         'size': [200,241],
                         'str': linear_str(15)},
    '16-cell linear 8': {'pt': [0.251572327044025,0.0555555555555556,0.072],
                         'size': [125,161],
                         'str': linear_str(16)},
    'H. juglandis': {'pt': [5.33333333333333,0.0622727272727273,0],
                     'size': [10622,10622],
                     'str': r'$\mathit{H. juglandis}$'},
    'fulvicephalus1 2': {'pt': [0.20125786163522,0.0565151515151515,0.08],
                         'size': [7,11],
                         'str': r'$\mathit{O. fulvicephalus}$'},
    'M15': {'pt': [5.33333333333333,0.0824242424242424,-0.02],
            'size': [14235,17408],
            'str': r'one-short'},
    'M12': {'pt': [6.08805031446541,0.0833838383838384,0.023],
            'size': [9644,14926],
            'str': r'$3/4$-pint'},
    'M20': {'pt': [6.08805031446541,0.0661111111111111,0.016],
            'size': [5876,6018],
            'str': r'$5/4$-pint'},
}

In [ ]:
def curated_scatter(regions_dict, xi=0, yi=1, ci=2, rescale_pulsevel=True):
    """
    Default: 
        0 = x = diffusion
        1 = y = vel
        2 = z = asymmetry
    """
    size_choice = 1  # pick 0 or 1 (0 = region, 1 = total)
    
    n = len(list(regions_dict.keys()))
    pts_arr = np.zeros((n, 3))
    size_arr = np.zeros(n)
    names = [0] * n
    
    i = 0
    for k, v in regions_dict.items():
        pts_arr[i, :] = v['pt']
        if rescale_pulsevel:
            a1 = 8
            pts_arr[i, 1] = pts_arr[i, 1] / 8
        size_arr[i] = v['size'][size_choice] 
        if v['str'] is None:
            names[i] = k
        else:
            names[i] = v['str']
            
        i += 1
    
    # size array transform here
    transform_size_log = True
    if transform_size_log:
        # log style
        sbase = 10
        #transform_size_arr = np.emath.logn(2, size_arr)
        transform_size_arr = np.log(size_arr)
        M1 = np.min(transform_size_arr)
        M2 = np.max(transform_size_arr)
    else:
        sbase = 100
        M1 = np.min(size_arr)
        M2 = np.max(size_arr)
        transform_size_arr = size_arr
        transform_size_arr = 0.5 + 2*(size_arr - M1) / (M2 - M1)  # will be sizes ranging from 0.5 to 3.5
    print('min, max', np.min(transform_size_arr), np.max(transform_size_arr))
    for idx in range(n):
        print(idx, names[idx], transform_size_arr[idx], size_arr[idx])
    
    fig = plt.figure(constrained_layout=False, figsize=(7, 4))
    gspec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[1, 0.035])
    ax0 = fig.add_subplot(gspec[0, 0])
    ax1 = fig.add_subplot(gspec[0, 1])
    
    pts_arr_rearranged = np.zeros_like(pts_arr)
    pts_arr_rearranged[:,0] = pts_arr[:, xi]
    pts_arr_rearranged[:,1] = pts_arr[:, yi]
    pts_arr_rearranged[:,2] = pts_arr[:, ci]
    
    fixed_cmap = plt.get_cmap('Spectral_r')  # viridis cividis RdYlBu coolwarm
    #fixed_cmap = plt.get_cmap('coolwarm')
    #fixed_cmap = plt.get_cmap('copper')
    
    if ci == 0:
        vmin = 0
        vmax = 7.0
    else:
        vmin = None
        vmax = None
    
    sc = ax0.scatter(pts_arr_rearranged[:,0], pts_arr_rearranged[:,1],
                     c=pts_arr_rearranged[:,2], 
                     s=sbase * transform_size_arr,
                     edgecolors='k',
                     linewidths=0.7,
                     cmap=fixed_cmap, vmin=vmin, vmax=vmax, 
                     zorder=5)

    for i in range(n):
        x, y, z_color = pts_arr_rearranged[i, :]
        label = names[i]
        ax0.text(x,y,s=label, zorder=10)
    
    ax0.grid(c='#e5e5e5', zorder=1)
    plt.colorbar(sc, cax=ax1)
    plt.savefig(DIR_OUTPUT + os.sep + "fig_4a_scatter2d_long.svg")
    plt.show()
    return

%matplotlib inline
curated_scatter(curated_regions, xi=2, yi=1, ci=0)

# Figure 5a,c and supplementary panels - 1D parameter scans
### - TODO: try diff num cell -> diff color point

In [ ]:
def plot_data_dot_by_dot_zorder(x, y, ax, z0=10, s0=6, lw=0.5, alpha=1.0, ec='#818285', fill='#d1d2d4', marker='o'):

    kw_data_marker_line = ''
    
    """
    sc_kwargs_exterior = dict(
        marker=marker,
        s=s0,
        c=ec,
        alpha=alpha,
    )
    sc_kwargs_interior = dict(
        marker=marker,
        s=s0/3.0,
        c=fill,
        alpha=alpha,
    )
    """
    sc_kwargs_exterior = dict(
        marker=marker,
        s=s0,
        c=fill,
        linewidths=lw,
        alpha=alpha,
        edgecolors=ec, 
        
    )
    
    n = len(x)
    assert n == len(y)
    
    # add line beneath scatter data
    #ax.plot(x, y, '--',  linewidth=0.5, **kw_data_marker_line, zorder=5)
    ax.plot(x, y, '--',  linewidth=0.5, c=ec, alpha=alpha, zorder=z0)

    zcount = z0
    """
    for i in range(n):
        ax.scatter(x[i], y[i], **sc_kwargs_exterior, zorder=zcount+ 1)
        ax.scatter(x[i], y[i], **sc_kwargs_interior, zorder=zcount + 2)
        zcount += 2
    """
    for i in range(n):
        ax.scatter(x[i], y[i], **sc_kwargs_exterior, zorder=zcount+ 1)
        #ax.scatter(x[i], y[i], **sc_kwargs_interior, zorder=zcount + 2)
        zcount += 2
    
    # add line beneath scatter data
    #ax.plot(x, y, '--',  linewidth=0.5, **kw_data_marker_line, zorder=5)
    
    return zcount

In [ ]:
import csv

def load_csv_x_y(fpath):
    with open(fpath) as f_csv:
        reader = csv.reader(f_csv, delimiter=",", quotechar='"')
        # next(reader, None)  # skip the headers
        data_read = [[float(i) for i in row] for row in reader]
    X = np.array(data_read)
    assert X.shape[1] == 2
    print('Read csv at %s' % fpath)
    print('Data shape:', X.shape)
    return X[:,0], X[:,1].astype(int) 

### Fig. 5a,c - load data and plot

In [ ]:
input_fig5a = PACKAGE_ROOT + os.sep + 'input' + os.sep + 'figure_data' + os.sep + 'fig5a'

# one curve to load - alternate method from np savetxt
fig5a_velocities_scaled = np.loadtxt(input_fig5a + os.sep + 'singlecell_ncycle_pulsev_scaled.txt')
fig5a_ncycle_data = np.loadtxt(input_fig5a + os.sep + 'singlecell_ncycle_simdata.txt')
fig5a_ncycle_heuristic = np.loadtxt(input_fig5a + os.sep + 'singlecell_ncycle_heuristic.txt')

In [ ]:
input_fig5c = PACKAGE_ROOT + os.sep + 'input' + os.sep + 'figure_data' + os.sep + 'fig5c'

# one curve to load, csv
fig5c_x, fig5c_y = load_csv_x_y(input_fig5c + os.sep + 'num_cells_1d_vary_diffusion_arg_oscillator_death.csv')

In [ ]:
input_fig5_supp_a = PACKAGE_ROOT + os.sep + 'input' + os.sep + 'figure_data' + os.sep + 'fig_supp_compare5a'

# two curves to load, both csv
fig_Fig5supp_a_curveA_x, fig_Fig5supp_a_curveA_y = load_csv_x_y(
    input_fig5_supp_a + os.sep + 'num_cells_1d_vary_pulse_vel_007.csv')
fig_Fig5supp_a_curveB_x, fig_Fig5supp_a_curveB_y = load_csv_x_y(
    input_fig5_supp_a + os.sep + 'num_cells_1d_vary_pulse_vel_narrow_dense.csv')

# rescale v axis by 1/a1
A1 = 8
fig_Fig5supp_a_curveA_x = fig_Fig5supp_a_curveA_x / A1
fig_Fig5supp_a_curveB_x = fig_Fig5supp_a_curveB_x / A1

In [ ]:
input_fig5_supp_b = PACKAGE_ROOT + os.sep + 'input' + os.sep + 'figure_data' + os.sep + 'fig_supp_compare5c'

# one curve to load, csv
fig_Fig5supp_b_x, fig_Fig5supp_b_y = load_csv_x_y(input_fig5_supp_b + os.sep + 'num_cells_1d_vary_diffusion_arg.csv')

In [ ]:
ax_lw = 0.5
figsize = (7, 1.5)

Make figure 5 (panel A and C)

In [ ]:
fig = plt.figure(constrained_layout=False, figsize=figsize)
gspec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[1, 1])
ax0 = fig.add_subplot(gspec[0, 0])
ax1 = fig.add_subplot(gspec[0, 1])

# make first panel Fig. 5a
zcount = plot_data_dot_by_dot_zorder(fig5a_velocities_scaled, fig5a_ncycle_data, ax0)

ax0.plot(fig5a_velocities_scaled, fig5a_ncycle_heuristic, '--s', label='heuristic', zorder=zcount + 1,
         markersize=0.5, c='#9E7BB5', linewidth=0.5)
# B768A2
ax0.legend()
ax0.set_ylabel(r'$n$ (num. cycles)')
ax0.set_xlabel('pulse velocity')
ax0.set_yticks([0,1,2,3,4,5])
ax0.set_xlim(0.0025, 0.027)
ax1.set_xlim(0, 5.4)

# make second panel Fig. 5c
zcount = plot_data_dot_by_dot_zorder(fig5c_x, fig5c_y, ax1)
ax1.set_yticks([0,4,8,12,16])
ax1.set_ylabel(r'$M$ (num. cells)')
ax1.set_xlabel('cell coupling')
ax1.set_xlim(0, 1.5)

for axis in ['top', 'bottom', 'left', 'right']:
    ax0.spines[axis].set_linewidth(ax_lw)
    ax1.spines[axis].set_linewidth(ax_lw)

ax0.grid(c='#e5e5e5', zorder=0, linewidth=0.5)
ax1.grid(c='#e5e5e5', zorder=0, linewidth=0.5)
plt.savefig(NB_OUTPUT + os.sep + "fig_5ac.svg")
plt.savefig(NB_OUTPUT + os.sep + "fig_5ac.pdf")
plt.show()

Make figure 5 supplementary figure (row of two panels, panel A and B)

In [ ]:
fig = plt.figure(constrained_layout=False, figsize=figsize)
gspec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[1, 1])
ax0 = fig.add_subplot(gspec[0, 0])
ax1 = fig.add_subplot(gspec[0, 1])

# make first panel a of Fig. 5supp
zcount = plot_data_dot_by_dot_zorder(fig_Fig5supp_a_curveB_x, fig_Fig5supp_a_curveB_y, ax0, 
                                     s0=12, ec='#c0c0c2', fill='#e8e8e9', alpha=1.0)

zcount = plot_data_dot_by_dot_zorder(fig_Fig5supp_a_curveA_x, fig_Fig5supp_a_curveA_y, ax0, 
                                     z0=zcount, s0=5, ec='#5072A7', fill='#4B9CD3', marker='^')
ax0.set_ylabel(r'$M$ (num. cells)')
ax0.set_xlabel('pulse velocity')
ax0.set_yticks([0,4,8,12,16])
ax0.set_xlim(0.01, 0.11/A1)

# make second panel Fig. 5c
zcount = plot_data_dot_by_dot_zorder(fig_Fig5supp_b_x, fig_Fig5supp_b_y, ax1)
#ax1.set_ylabel(r'$M$ (num. cells)')
ax1.set_xlabel('cell coupling')
ax1.set_yticks([8,10,12,14,16])
ax1.set_xlim(0, 0.16)

for axis in ['top', 'bottom', 'left', 'right']:
    ax0.spines[axis].set_linewidth(ax_lw)
    ax1.spines[axis].set_linewidth(ax_lw)

ax0.grid(c='#e5e5e5', zorder=0, linewidth=0.5)
ax1.grid(c='#e5e5e5', zorder=0, linewidth=0.5)
plt.savefig(NB_OUTPUT + os.sep + "fig_5supp.svg")
plt.savefig(NB_OUTPUT + os.sep + "fig_5supp.pdf")
plt.show()
